In [1]:
import parse_output
import pandas as pd

In [2]:
def was_traceroute_successful(traceroute):
    traceroute_ips = traceroute.filter(regex=".*ip.*")
    """Returns true if we found the resolved IP in the traceroute (success)."""
    ips = list(traceroute_ips)
    dest = list(traceroute_ips)[0]
    return dest in ips[1:]

In [7]:
def get_final_rtt(traceroute_rtts):
    final_rtt = None
    for rtt in traceroute_rtts:
        try:
            final_rtt = float(rtt)
        except:
            pass
    assert final_rtt != None
    return final_rtt 

In [14]:
def cleanup_trees_for_rtt_analysis(trees):
    # Only consider successful rtts
    print(f'{len(trees)} total traceroutes.')
    successful_rtt_trees = trees[trees.apply(lambda x: was_traceroute_successful(x), axis=1)]

    # We only care about the rtts
    print(f'{len(successful_rtt_trees)} fully successful traceroutes.')
    rtt_trees = successful_rtt_trees.filter(regex=".*rtt.*|dest name")

    # We only want the final rtt we observed
    rtt_trees["final_rtt"] = rtt_trees.apply(lambda row: get_final_rtt(row), axis=1)

    # Simplified into two columns
    simplified_rtts = rtt_trees[["dest name", "final_rtt"]]

    return simplified_rtts

In [15]:
aws_ohio_data_filepath = "data/aws-ohio-results-1637264695-incomplete.csv"
aws_ohio_trees = parse_output.parse_output(aws_ohio_data_filepath)
aws_ohio_simplified_rtts = cleanup_trees_for_rtt_analysis(aws_ohio_trees)

home_victor_data_filepath = "data/home-victor-results-1637268720-incomplete.csv"
home_victor_trees = parse_output.parse_output(home_victor_data_filepath)
home_victor_simplified_rtts = cleanup_trees_for_rtt_analysis(home_victor_trees)

12348 total traceroutes.
3009 fully successful traceroutes.


/tmp/ipykernel_370/938413540.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtt_trees["final_rtt"] = rtt_trees.apply(lambda row: get_final_rtt(row), axis=1)


9665 total traceroutes.
3734 fully successful traceroutes.


,final_rtt
count,3734.000000
mean,38.652097
std,52.049089
min,8.754000
25%,15.918250
50%,19.495000
75%,35.009750
max,248.438000


In [19]:
aws_ohio_simplified_rtts \
    .groupby("dest name") \
    .describe() \
    

#home_victor_simplified_rtts.describe())

final_rtt                                                  \
                   count       mean       std     min       25%      50%   
dest name                                                                  
adobe.com            2.0  10.880000  0.004243  10.877  10.87850  10.8800   
apple.com          252.0  11.404603  0.346304  10.784  11.13900  11.3725   
canva.com          193.0  11.058166  0.187364  10.794  10.97900  11.0250   
chaturbate.com     252.0  10.959028  0.156041  10.694  10.87625  10.9390   
craigslist.com     209.0  77.427397  0.373523  76.650  77.21300  77.4990   
dropbox.com        252.0  10.975444  0.223969  10.756  10.88600  10.9365   
espn.com           244.0  18.847885  4.123366   7.837  20.12800  20.1680   
facebook.com       252.0  10.801857  0.220362  10.597  10.72000  10.7790   
homedepot.com      111.0  11.133144  0.192737  10.896  11.00800  11.1010   
instagram.com      252.0  10.810968  0.230447  10.572  10.71775  10.7690   
myshopify.com      184.0  11.053527  0.177666  10.790  10.95550  11.0460   
tiktok.com         246.0  18.463813  4.248926   7.780  20.12625  20.1710   
twitter.com        252.0  30.830615  4.593135  21.840  28.81425  29.3105   
youtube.com         61.0   9.144836  0.719506   8.037   8.35600   9.1460   
zillow.com         247.0  18.617024  4.179583   7.824  20.13500  20.1760   

                                  
                     75%     max  
dest name                         
adobe.com       10.88150  10.883  
apple.com       11.64300  12.756  
canva.com       11.07200  12.177  
chaturbate.com  11.00900  12.033  
craigslist.com  77.61600  78.770  
dropbox.com     10.99350  13.481  
espn.com        20.21600  34.629  
facebook.com    10.84700  13.915  
homedepot.com   11.19650  12.161  
instagram.com   10.85475  13.484  
myshopify.com   11.11875  12.486  
tiktok.com      20.21875  21.290  
twitter.com     35.71550  43.918  
youtube.com      9.36200  10.499  
zillow.com      20.22850  23.354